## 1. Setup & Check GPU

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU! Go to Runtime → Change runtime type → GPU")

In [ ]:
# Install Ultralytics YOLOv8
!pip install -q ultralytics

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Configuration

In [ ]:
# ============ CONFIG ============
import os

# Dataset paths (adjust to your Google Drive path)
DATASET_ROOT = "/content/drive/MyDrive/processed2"

# Training settings
MODEL_SIZE = 'n'      # 'n'=nano, 's'=small, 'm'=medium, 'l'=large, 'x'=xlarge
EPOCHS = 100          # YOLOv8 typically needs more epochs
IMG_SIZE = 640        # Image size
BATCH_SIZE = 16       # Adjust based on GPU memory
PATIENCE = 20         # Early stopping patience

# Class names
CLASS_NAMES = [
    'Longitudinal_crack',   # D00
    'Transverse_crack',     # D10  
    'Alligator_crack',      # D20
    'Other_damage',         # Other
     'Pothole'              # D40
]
NUM_CLASSES = len(CLASS_NAMES)

# Output directory
OUTPUT_DIR = "/content/yolov8_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Model: YOLOv8{MODEL_SIZE}")
print(f"Epochs: {EPOCHS}, Batch: {BATCH_SIZE}, ImgSize: {IMG_SIZE}")
print(f"Classes: {NUM_CLASSES} - {CLASS_NAMES}")

## 4. Create Dataset YAML
YOLOv8 requires a YAML file describing the dataset structure.

In [ ]:
# Verify dataset exists
train_images = os.path.join(DATASET_ROOT, 'train', 'images')
val_images = os.path.join(DATASET_ROOT, 'val', 'images')
test_images = os.path.join(DATASET_ROOT, 'test', 'images')

print("📂 Dataset structure:")
for split, path in [('Train', train_images), ('Val', val_images), ('Test', test_images)]:
    if os.path.exists(path):
        count = len([f for f in os.listdir(path) if f.endswith(('.jpg', '.png', '.jpeg'))])
        print(f"  {split}: {count} images")
    else:
        print(f"  {split}: NOT FOUND at {path}")

In [ ]:
# Create dataset.yaml for YOLOv8
import yaml

dataset_yaml = {
    'path': DATASET_ROOT,
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'names': {i: name for i, name in enumerate(CLASS_NAMES)},
    'nc': NUM_CLASSES
}

yaml_path = os.path.join(OUTPUT_DIR, 'dataset.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

print(f"✓ Created {yaml_path}")
print("\nContents:")
with open(yaml_path, 'r') as f:
    print(f.read())

In [ ]:
# ============ VERIFY & CLEAN DATASET ============
# Remove images without labels and labels without images
import os

def clean_split(split_name, images_dir, labels_dir, dry_run=False):
    """Remove orphan images/labels in a split."""
    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        print(f"  ⚠️ {split_name}: directories not found, skipping")
        return 0, 0
    
    # Get file basenames (without extension)
    images = {os.path.splitext(f)[0]: f for f in os.listdir(images_dir) 
              if f.lower().endswith(('.jpg', '.png', '.jpeg'))}
    labels = {os.path.splitext(f)[0]: f for f in os.listdir(labels_dir) 
              if f.lower().endswith('.txt')}
    
    # Find orphans
    images_without_labels = set(images.keys()) - set(labels.keys())
    labels_without_images = set(labels.keys()) - set(images.keys())
    
    removed_images = 0
    removed_labels = 0
    
    # Remove orphan images
    for basename in images_without_labels:
        img_path = os.path.join(images_dir, images[basename])
        if dry_run:
            print(f"    [DRY RUN] Would remove image: {images[basename]}")
        else:
            os.remove(img_path)
            print(f"    🗑️ Removed image: {images[basename]}")
        removed_images += 1
    
    # Remove orphan labels
    for basename in labels_without_images:
        label_path = os.path.join(labels_dir, labels[basename])
        if dry_run:
            print(f"    [DRY RUN] Would remove label: {labels[basename]}")
        else:
            os.remove(label_path)
            print(f"    🗑️ Removed label: {labels[basename]}")
        removed_labels += 1
    
    valid_pairs = len(set(images.keys()) & set(labels.keys()))
    print(f"  {split_name}: {valid_pairs} valid pairs | removed {removed_images} images, {removed_labels} labels")
    return removed_images, removed_labels

print("🔍 Checking dataset for orphan files...\n")

# Set to False to actually delete files, True to just preview
DRY_RUN = False  # ⚠️ Set to False to actually remove files!

total_img, total_label = 0, 0

for split in ['train', 'val', 'test']:
    img_dir = f"{DATASET_ROOT}/{split}/images"
    label_dir = f"{DATASET_ROOT}/{split}/labels"
    ri, rl = clean_split(split, img_dir, label_dir, dry_run=DRY_RUN)
    total_img += ri
    total_label += rl

print(f"\n{'[DRY RUN] ' if DRY_RUN else ''}Total removed: {total_img} images, {total_label} labels")
if DRY_RUN:
    print("⚠️ Set DRY_RUN = False and re-run to actually delete files")

## 5. Check Label Distribution

In [ ]:
from collections import Counter
import numpy as np

def analyze_labels(labels_dir, max_files=None):
    """Analyze label distribution."""
    if not os.path.exists(labels_dir):
        print(f"  ⚠️ Not found: {labels_dir}")
        return
    
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    if max_files:
        label_files = label_files[:max_files]
    
    class_counts = Counter()
    box_sizes = []
    empty_count = 0
    
    for lf in label_files:
        with open(os.path.join(labels_dir, lf), 'r') as f:
            lines = f.readlines()
        
        if len(lines) == 0:
            empty_count += 1
            continue
            
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls_id = int(parts[0])
                w, h = float(parts[3]), float(parts[4])
                class_counts[cls_id] += 1
                box_sizes.append(w * h)
    
    print(f"  Files: {len(label_files)} ({empty_count} empty)")
    print(f"  Total boxes: {sum(class_counts.values())}")
    if box_sizes:
        print(f"  Avg box size: {np.mean(box_sizes)*100:.1f}% of image")
    print(f"  Class distribution:")
    for cls_id in sorted(class_counts.keys()):
        name = CLASS_NAMES[cls_id] if cls_id < len(CLASS_NAMES) else f'Unknown({cls_id})'
        print(f"    {cls_id}: {class_counts[cls_id]:5d} - {name}")

print("📊 Train labels:")
analyze_labels(os.path.join(DATASET_ROOT, 'train', 'labels'))
print("\n📊 Val labels:")
analyze_labels(os.path.join(DATASET_ROOT, 'val', 'labels'))

## 6. Train YOLOv8

In [ ]:
from ultralytics import YOLO

# Load pretrained YOLOv8 model
model = YOLO(f'yolov8{MODEL_SIZE}.pt')

print(f"✓ Loaded YOLOv8{MODEL_SIZE} with pretrained COCO weights")
print(f"  This provides a strong starting point for fine-tuning!")

In [ ]:
# Train the model
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    save=True,
    device=0,  # Use GPU 0
    workers=2,
    project=OUTPUT_DIR,
    name='train',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,  # Final LR = lr0 * lrf
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    box=7.5,       # Box loss gain
    cls=0.5,       # Cls loss gain
    dfl=1.5,       # DFL loss gain
    hsv_h=0.015,   # HSV-Hue augmentation
    hsv_s=0.7,     # HSV-Saturation augmentation
    hsv_v=0.4,     # HSV-Value augmentation
    degrees=10.0,  # Rotation augmentation
    translate=0.1, # Translation augmentation
    scale=0.5,     # Scale augmentation
    fliplr=0.5,    # Horizontal flip
    mosaic=1.0,    # Mosaic augmentation
    mixup=0.1,     # Mixup augmentation
    copy_paste=0.1,# Copy-paste augmentation
    verbose=True
)

print("\n🎉 Training complete!")

In [ ]:
# Backup to Google Drive immediately after training
import shutil

# Define best model path
best_model_path = os.path.join(OUTPUT_DIR, 'train', 'weights', 'best.pt')

drive_backup = "/content/drive/MyDrive/yolov8_backup"
os.makedirs(drive_backup, exist_ok=True)
shutil.copy(best_model_path, drive_backup)
print(f"✓ Model backed up to {drive_backup}")

## 7. Evaluate on Validation Set

In [ ]:
# Load best model
best_model_path = os.path.join(OUTPUT_DIR, 'train', 'weights', 'best.pt')
best_model = YOLO(best_model_path)

# Evaluate
metrics = best_model.val(data=yaml_path, imgsz=IMG_SIZE, batch=BATCH_SIZE)

print("\n📊 Validation Results:")
print(f"  mAP@0.5:      {metrics.box.map50:.4f}")
print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"  Precision:    {metrics.box.mp:.4f}")
print(f"  Recall:       {metrics.box.mr:.4f}")

In [ ]:
# Per-class metrics
print("\n📊 Per-class AP@0.5:")
for i, name in enumerate(CLASS_NAMES):
    if i < len(metrics.box.ap50):
        print(f"  {name}: {metrics.box.ap50[i]:.4f}")

## 8. Visualize Predictions

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random

# Get validation images
val_images_list = [f for f in os.listdir(val_images) if f.endswith(('.jpg', '.png', '.jpeg'))]
sample_images = random.sample(val_images_list, min(6, len(val_images_list)))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for ax, img_name in zip(axes.flat, sample_images):
    img_path = os.path.join(val_images, img_name)
    
    # Run prediction
    results = best_model.predict(img_path, conf=0.25, imgsz=IMG_SIZE, verbose=False)
    
    # Plot with boxes
    result_img = results[0].plot()
    ax.imshow(result_img[:, :, ::-1])  # BGR to RGB
    
    n_boxes = len(results[0].boxes) if results[0].boxes is not None else 0
    ax.set_title(f'{img_name}\n({n_boxes} detections)')
    ax.axis('off')

plt.suptitle('YOLOv8 Road Damage Detection Results', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'predictions.png'), dpi=150)
plt.show()

## 9. View Training Curves

In [ ]:
# Display training curves from results.csv
import pandas as pd

results_csv = os.path.join(OUTPUT_DIR, 'train', 'results.csv')
if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()  # Remove whitespace from column names
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Box loss
    axes[0, 0].plot(df['train/box_loss'], label='Train')
    axes[0, 0].plot(df['val/box_loss'], label='Val')
    axes[0, 0].set_title('Box Loss')
    axes[0, 0].legend()
    
    # Classification loss
    axes[0, 1].plot(df['train/cls_loss'], label='Train')
    axes[0, 1].plot(df['val/cls_loss'], label='Val')
    axes[0, 1].set_title('Classification Loss')
    axes[0, 1].legend()
    
    # mAP
    axes[1, 0].plot(df['metrics/mAP50(B)'], label='mAP@0.5')
    axes[1, 0].plot(df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95')
    axes[1, 0].set_title('mAP')
    axes[1, 0].legend()
    
    # Precision & Recall
    axes[1, 1].plot(df['metrics/precision(B)'], label='Precision')
    axes[1, 1].plot(df['metrics/recall(B)'], label='Recall')
    axes[1, 1].set_title('Precision & Recall')
    axes[1, 1].legend()
    
    for ax in axes.flat:
        ax.set_xlabel('Epoch')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'training_curves.png'), dpi=150)
    plt.show()
    
    # Print best metrics
    print(f"\n🏆 Best Results:")
    print(f"  Best mAP@0.5:      {df['metrics/mAP50(B)'].max():.4f}")
    print(f"  Best mAP@0.5:0.95: {df['metrics/mAP50-95(B)'].max():.4f}")
    print(f"  Best Precision:    {df['metrics/precision(B)'].max():.4f}")
    print(f"  Best Recall:       {df['metrics/recall(B)'].max():.4f}")
else:
    print(f"⚠️ Results file not found: {results_csv}")

## 10. Test on Test Set (Optional)

In [ ]:
# Run inference on test set
test_results = best_model.predict(
    source=test_images,
    conf=0.25,
    imgsz=IMG_SIZE,
    save=True,
    save_txt=True,
    project=OUTPUT_DIR,
    name='test_predictions',
    exist_ok=True
)

print(f"\n✓ Test predictions saved to {OUTPUT_DIR}/test_predictions/")

## 11. Export Model

In [ ]:
# Export to different formats
print("📦 Exporting model...")

# Export to ONNX (for deployment)
best_model.export(format='onnx', imgsz=IMG_SIZE)
print("  ✓ Exported to ONNX")

# Copy best weights to outputs
import shutil
final_model_path = os.path.join(OUTPUT_DIR, 'yolov8_road_damage_best.pt')
shutil.copy(best_model_path, final_model_path)
print(f"  ✓ Best weights saved to {final_model_path}")

## 12. Download Model

In [ ]:
from google.colab import files

# Download best model
files.download(final_model_path)
print("✓ Model downloaded!")

## Summary

This notebook uses **Ultralytics YOLOv8** which provides:
- ✅ Pretrained COCO weights for transfer learning
- ✅ State-of-the-art detection architecture
- ✅ Built-in augmentations (mosaic, mixup, copy-paste)
- ✅ Automatic learning rate scheduling
- ✅ Early stopping
- ✅ Built-in metrics (mAP, precision, recall)
- ✅ Easy export to ONNX/TensorRT for deployment

**Expected mAP@0.5 for road damage:** 0.40-0.65 depending on dataset quality.